In [1]:
#!/usr/bin/env python

import math
import healpy as hp
from classy import Class
from scipy.stats import norm
import numpy as np
import sys
import time
sys.path.insert(1, '../../../PySSC/')
sys.path.insert(1, '../../../CLASS/')
import PySSC

try:
    import gi

    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())


# Initialize the `Cosmological model`

In [2]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0  =  67.81
cosmo.props.Omegac  =  0.2612
cosmo.props.Omegab =  0.0486
cosmo.props.Omegax = 1 - 0.0486 -  0.2612
cosmo.props.w = -1.0


reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 



cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()
prim.props.n_SA = 0.9660
print(0.82505858,cosmo.sigma8(psf),cosmo.Omega_k0())

old_amplitude = math.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log ((0.82505858/ cosmo.sigma8(psf))**2 * old_amplitude)
print(0.82505858, cosmo.sigma8(psf))

z_obs  = Ncm.Vector.new_array(np.linspace(0.1,0.8,8))
lnM_obs = Ncm.Vector.new_array(np.linspace(np.log(10)*13.0,np.log(10)*15, 2))

0.82505858 0.8900071895091025 -9.085228043836935e-05
0.82505858 0.82505858


# Construct the $S_{ij}$  Matrix

`kernel and masks`

In [3]:
cosmo_fid = Class()
cosmo_fid.set({'h':cosmo.props.H0/100 ,'Omega_cdm':cosmo.props.Omegac ,'Omega_b':cosmo.props.Omegab ,'sigma8':0.82505858,'n_s':0.9660,'output':'mPk'})
cosmo_fid.compute()
nz       = 400
z_arr    = np.linspace(0,1.9,num=nz+1)[1:]


nbins_T   = z_obs.len() -1


kernels_T = np.zeros((nbins_T,nz))
for i in range(nbins_T):
    zminbin = z_obs.get(i) ; zmaxbin = z_obs.get(i+1) ; Dz = zmaxbin-zminbin
    for iz in range(nz):
        z = z_arr[iz]
        if ((z>zminbin) and (z<=zmaxbin)):
            kernels_T[i,iz] = 1/Dz




NSIDE = 2*512
NPIX = hp.nside2npix(NSIDE)
m = np.zeros(hp.nside2npix(NSIDE))
pix_theta_ecl, pix_phi_ecl = hp.pix2ang(NSIDE, np.arange(NPIX))

#Guaranteed mask
mask1_guaranteed = np.zeros(NPIX)
mask2_guaranteed = np.zeros(NPIX)
mask3_guaranteed = np.zeros(NPIX)

mask1_guaranteed_condition = (pix_phi_ecl > 3.*np.pi/4.) & (pix_phi_ecl < 13.*np.pi/12.) & (pix_theta_ecl < np.pi/2. - 30.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_guaranteed_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)
mask3_guaranteed_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)

mask1_guaranteed[mask1_guaranteed_condition] = 1
mask2_guaranteed[mask2_guaranteed_condition] = 1
mask3_guaranteed[mask3_guaranteed_condition] = 1

mask_guaranteed = mask1_guaranteed + mask2_guaranteed + mask3_guaranteed

#Total mask
mask1_full = np.zeros(NPIX)
mask2_full = np.zeros(NPIX)
mask3_full = np.zeros(NPIX)

mask1_full_condition = (pix_phi_ecl > 2.*np.pi/3.) & (pix_phi_ecl < 3.*np.pi/2.) & (pix_theta_ecl < np.pi/2. - 10.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_full_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)
mask3_full_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)

mask1_full[mask1_full_condition] = 1
mask2_full[mask2_full_condition] = 1
mask3_full[mask3_full_condition] = 1

mask_full = mask1_full + mask2_full + mask3_full






`S matrices`

In [4]:
#Full sky
t0 = time.time()
S_lacasa = PySSC.Sij(z_arr,kernels_T,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Full sky time (seconds) = ' + str(t1-t0))


S_full = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_full.set(i,j, S_lacasa[i][j])


#Partial-Sky guaranteed
t0 = time.time()
S_lacasa = PySSC.Sij_psky(z_arr,kernels_T,mask=mask_guaranteed,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Partial sky guaranteed time (seconds) = ' + str(t1-t0))

S_guaranteed = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_guaranteed.set(i,j, S_lacasa[i][j])


#Partial-Sky maximal
t0 = time.time()
S_lacasa = PySSC.Sij_psky(z_arr,kernels_T,mask=mask_full,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Partial sky maximal time (seconds) = ' + str(t1-t0))

S_maximal = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_maximal.set(i,j, S_lacasa[i][j])

Full sky time (seconds) = 1.0035312175750732
Partial sky guaranteed time (seconds) = 51.21904253959656
Partial sky maximal time (seconds) = 30.05237317085266


# Define proxy modelling


In [5]:
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e14),math.log(10)*np.log10(1e16)))
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,2))

# Initialize the `ClusterAbundance` object

In [6]:
mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf,hbias_Tinker)

# Data model

$f(x_i)$ for the Gaussian distribution $$-2\ln L = \sum_{ij}\left[(f(x_i) - y_i)C^{-1}_{ij}(f(x_j) - y_j)\right] + \dots$$

In [7]:
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])


ncounts_gauss = Nc.DataClusterNCountsGauss.new(cad)
ncounts_gauss.set_size((z_obs.len()-1)*(lnM_obs.len()-1))
ncounts_gauss.set_init(True)
ncounts_gauss.use_norma(True)
ncounts_gauss.set_z_obs(z_obs)
ncounts_gauss.set_lnM_obs(lnM_obs)
rng = Ncm.RNG.seeded_new (None, 42)

# Creating the fit object and calculating the Fisher Matrix

`No SSC`

In [8]:
print('No SSC')
guaranteed_area = 2959.1
cad.set_area(guaranteed_area*(np.pi/180)**2)

dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()


lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

t0 = time.time()
fit.log_info()
fit.fisher()
fit.log_covar ()
t1 = time.time()
print('No SSC time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

ncounts_gauss.resample(mset, rng)
t0 = time.time()
fit.obs_fisher()
fit.log_covar ()
t1 = time.time()
print('No SSC obs maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

No SSC
0.82505858
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.81               [FIXED]
#   -     Omegac[01]:  0.2612              [FREE]
#   -     Omegax[02]:  0.6902              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FIXED]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#-------------------------------------------------

`Full-Sky`

In [9]:
ncounts_gauss.set_has_ssc(True)
print('Full Sky')
guaranteed_area = 2959.1
cad.set_area(guaranteed_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_full)

dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()


lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

t0 = time.time()
fit.log_info()
fit.fisher()
fit.log_covar ()
t1 = time.time()
print('Full sky maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")


ncounts_gauss.resample(mset, rng)
t0 = time.time()
fit.obs_fisher()
fit.log_covar ()
t1 = time.time()
print('Full sky obs maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Full Sky


0.82505858
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.81               [FIXED]
#   -     Omegac[01]:  0.2612              [FREE]
#   -     Omegax[02]:  0.6902              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FIXED]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#--------------------------------------------------------

`Partial-Sky guaranteed`

In [11]:
print('Partial Sky guaranteed area')
guaranteed_area = 2959.1
cad.set_area(guaranteed_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_guaranteed)




dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)

cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = False
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

t0 = time.time()
fit.log_info()
fit.fisher()
fit.log_covar ()
t1 = time.time()
print('Partial sky guaranteed time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

ncounts_gauss.resample(mset, rng)
t0 = time.time()
fit.obs_fisher()
fit.log_covar ()
t1 = time.time()
print('Partial sky obs guaranteed time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Partial Sky guaranteed area
0.82505858
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.81               [FIXED]
#   -     Omegac[01]:  0.2612              [FREE]
#   -     Omegax[02]:  0.6902              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FIXED]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#----------------------------


(process:7490): NUMCOSMO-WARNING **: 12:03:29.971: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


`Partial-Sky Maximal`

In [10]:
print('Partial Sky maximal area')
maximal_area = 10009.97
cad.set_area(maximal_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_maximal)




dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)

cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = False
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

t0 = time.time()
fit.log_info()
fit.fisher()
fit.log_covar ()
t1 = time.time()
print('Partial sky maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

ncounts_gauss.resample(mset, rng)
t0 = time.time()
fit.obs_fisher()
fit.log_covar ()
t1 = time.time()
print('Partial sky obs maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Partial Sky maximal area
0.82505858
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.81               [FIXED]
#   -     Omegac[01]:  0.2612              [FREE]
#   -     Omegax[02]:  0.6902              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FIXED]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#-------------------------------